# Model Training

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import tifffile as tiff

## Load training and label

In [ ]:
# DATOS_train
datos_train = pd.read_csv("data/DATOS_train.csv")

In [ ]:
path_feats = "data/train_features/content/train_features/"
path_labels = "data/train_labels/content/train_labels/"

In [ ]:
feats = []
labels = []
for n in range(len(datos_train)):
    path_feat = datos_train.iloc[n].feature_image
    path_label = datos_train.iloc[n].label_image
    feat = tiff.imread(path_feats + path_feat)
    label = tiff.imread(path_labels + path_label)
    
    feats.append(feat)
    labels.append(label)
    
feats = np.array(feats)
labels = np.array(labels)

In [ ]:
feats.shape, labels.shape

In [ ]:
feats.min(), feats.max()

In [ ]:
labels.min(), labels.max()

In [ ]:
from collections import Counter
Counter(labels.flatten())

## Prepocess Features

In [ ]:
# np.min(feats[1][:, :, 1])

In [ ]:
# # distribution per layer

# for i in range(10):
#     mins = []
#     maxs = []
#     for j in range(feats.shape[0]):
#         vals = feats[j][:, :, i].flatten()
#         min_ = vals.min()
#         max_ = vals.max()
    
#         mins.append(min_)
#         maxs.append(max_)

#     f, ax = plt.subplots(1, 2, figsize=(10, 3))
#     ax[0].hist(mins)
#     ax[0].set_title(f'Mins for layer {i}')
#     ax[1].hist(maxs)
#     ax[1].set_title(f'Maxs for layer {i}')
#     plt.show()
    
    

In [ ]:
# # which have minimum values < - 1?
# outliers = []
# for i in feats:
#     if np.min(i) < -1:
#         outliers.append(i)

In [ ]:
# len(outliers)

In [ ]:
# we remove outliers
feats_final = []
labels_final0 = []

for n in range(len(datos_train)):
    path_feat = datos_train.iloc[n].feature_image
    path_label = datos_train.iloc[n].label_image
    
    feat = tiff.imread(path_feats + path_feat)
    label = tiff.imread(path_labels + path_label)
    
    if feat.min() >= -1:
        feats_final.append(feat)
        labels_final0.append(label)
    
feats_final = np.array(feats_final)
labels_final0 = np.array(labels_final0)

In [ ]:
feats_final.shape, labels_final0.shape

In [ ]:
# # distribution per layer

# for i in range(10):
#     mins = []
#     maxs = []
#     for j in range(feats_final.shape[0]):
#         vals = feats_final[j][:, :, i].flatten()
#         min_ = vals.min()
#         max_ = vals.max()
    
#         mins.append(min_)
#         maxs.append(max_)

#     f, ax = plt.subplots(1, 2, figsize=(10, 3))
#     ax[0].hist(mins)
#     ax[0].set_title(f'Mins for layer {i}')
#     ax[1].hist(maxs)
#     ax[1].set_title(f'Maxs for layer {i}')
#     plt.show()
    
    

In [ ]:
labels_final0[0].shape
labels_final0[0]

## Preprocess label - scale

In [ ]:
# labels_final = labels_final0/7
# labels_final.min(), labels_final.max()

## Preprocess label (separate per class)
Optional. We split each prediction into a 128 X 128 X 8 tensor. Each of the 8 layers correspond to classes 0, 1, 2, 4, 5, 6, 7, 15.

In [ ]:
sample_feat = feats_final[20]
sample_feat.shape

In [ ]:
plt.imshow(sample_feat[:,:,3])

In [ ]:
sample_label = labels_final0[20]
sample_label.shape

In [ ]:
plt.imshow(sample_label)

In [ ]:
def split_array(sample_label):
    new_array = []

    for i in [0, 1, 2, 4, 5, 6, 7, 15]:
        layer = sample_label==i
        layer = layer.astype(int)
        new_array.append(layer)
    new_array = np.array(new_array)
    new_array = np.swapaxes(new_array, 0, 2)
    new_array = np.rot90(new_array, k=-1)
    new_array = np.fliplr(new_array)
    
    
    return new_array

In [ ]:
sample_split = split_array(sample_label)
for i in range(8):
    plt.imshow(sample_split[:, :, i])
    plt.show()

In [ ]:
labels_final = np.array([split_array(i) for i in labels_final0])

In [ ]:
labels_final.shape

In [ ]:
feats_final.shape, labels_final.shape

In [ ]:
## Compress labels

def compress_label(matrix):
    classes = [0, 1, 2, 4, 5, 6, 7, 15]

    new_array = []
    for i in range(8):
        layer = matrix[:, :, i]
        layer = np.round(layer) * classes[i]
        new_array.append(layer)
    
    new_array = np.array(new_array)
    new_array = np.swapaxes(new_array, 0, 2)
    new_array = np.max(new_array, axis=2)
    new_array = np.rot90(new_array, k=-1)
    new_array = np.fliplr(new_array)
    
    return new_array

In [ ]:
plt.imshow(labels_final0[20])

In [ ]:
plt.imshow(compress_label(labels_final[20]))

# Model Definitions

In [ ]:
# example of defining a 70x70 patchgan discriminator model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import Dropout

from PIL import Image

### Encoder and Decoder Definition

In [ ]:
# define an encoder block
def define_encoder_block(layer_in, n_filters, batchnorm=True):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # add downsampling layer
    g = Conv2D(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
    # conditionally add batch normalization
    if batchnorm:
        g = BatchNormalization()(g, training=True)
    # leaky relu activation
    g = LeakyReLU(alpha=0.2)(g)
    return g
 
# define a decoder block
def decoder_block(layer_in, skip_in, n_filters, dropout=True):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # add upsampling layer
    g = Conv2DTranspose(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
    # add batch normalization
    g = BatchNormalization()(g, training=True)
    # conditionally add dropout
    if dropout:
        g = Dropout(0.5)(g, training=True)
    # merge with skip connection
    g = Concatenate()([g, skip_in])
    # relu activation
    g = Activation('relu')(g)
    return g

In [ ]:
activation='sigmoid'

# define the standalone generator model
def define_generator(image_shape=(128,128,10)):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # image input
    in_image = Input(shape=image_shape)
    # encoder model: C64-C128-C256
    e1 = define_encoder_block(in_image, 64, batchnorm=False)
    e2 = define_encoder_block(e1, 128)
    e3 = define_encoder_block(e2, 256)
    # bottleneck, no batch norm and relu
    b = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(e3)
    b = Activation('relu')(b)
    # decoder model: CD512-CD1024-CD1024-C1024-C1024-C512-C256-C128
    d1 = decoder_block(b, e3, 256, dropout=False)
    d2 = decoder_block(d1, e2, 128, dropout=False)
    d3 = decoder_block(d2, e1, 64, dropout=False)
    # output
    g = Conv2DTranspose(8, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d3)
    out_image = Activation(activation)(g)
    # define model
    model = Model(in_image, out_image)
    return model

### Define Sample Generation

In [ ]:
# select a batch of random samples, returns images and target
def generate_real_samples(dataset, n_samples):
    # unpack dataset
    trainA, trainB = dataset
    # choose random instances
    ix = np.random.randint(0, trainA.shape[0], n_samples)
    # retrieve selected images
    X1, X2 = trainA[ix], trainB[ix]
    
    return [X1, X2]

### Date for Naming

In [ ]:
from datetime import date
today = str(date.today().year)+'_'+str(date.today().month)+'_'+str(date.today().day)

### Standard U-Net

In [ ]:
# define image shape
image_shape = (128,128,10)
# create the model
unet_model = define_generator(image_shape)

opt = Adam(lr=0.0002, beta_1=0.5)
# unet_model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[100])

loss = 'categorical_crossentropy'
unet_model.compile(loss=loss, optimizer=opt, loss_weights=[100])

# summarize the model
unet_model.summary()

### U-Net with Adversarial Loss on Prediction Network

### WGANs

## Training
We measure:
<br>1) Average of intersection over Union for each of the classes
<br>2) Pixel-by-pixel accuracy

In [ ]:
# # k-fold cross validation function

# def get_train_valid(num_samples, k):
#     valid_inds = []
#     train_inds = []
    
#     num_valid = num_samples // k
#     num_train = num_samples - num_valid
    
#     for i in range(k):
#         valid_ind = np.arange(num_valid * i, num_valid * i + num_valid)
#         train_ind = np.array([i for i in np.arange(num_samples) if i not in valid_ind])
        
#         valid_inds.append(valid_ind)
#         train_inds.append(train_ind)
        
#     return valid_inds, train_inds
# #         print(valid_inds[0], valid_inds[-1])
# #         print(len(valid_inds))
# #         print(len(train_inds))

### Standard U-net

In [ ]:
# train pix2pix models
def train(g_model, dataset, n_epochs=1, n_batch=1):
    # unpack dataset
    trainA, trainB = dataset
    # calculate the number of batches per training epoch
    bat_per_epo = int(len(trainA) / n_batch)
    # calculate the number of training iterations
    n_steps = bat_per_epo * n_epochs
    print(n_steps)
    # manually enumerate epochs
    for i in range(n_steps):
        # select a batch of real samples
        [X_realA, X_realB] = generate_real_samples(dataset, n_batch)
        # update the generator
        g_loss = g_model.train_on_batch(X_realA, X_realB)
        # summarize performance
        if i % 1000 == 0:
            print('>%d, g[%.3f]' % (i+1, g_loss))

In [ ]:
# # Optional Cross Validation
# num_samples = feats_final.shape[0]
# k = 3
# valid_inds, train_inds = get_train_valid(num_samples, k)

# i = 0
# train_feats = feats_final[train_inds[i]]
# valid_feats = feats_final[valid_inds[i]]

# train_labels = labels_final[train_inds[i]]
# valid_labels = labels_final[valid_inds[i]]

# train_feats.shape, valid_feats.shape, train_labels.shape, valid_labels.shape

train_feats = feats_final
train_labels = labels_final

train_feats.shape, train_labels.shape

In [ ]:
file = 'datos_unet_gen_categorical_crossentropy_sigmoid_2019_11_17_0'
unet_model.load_weights(f"models/{file}.h5")

In [ ]:
# # augmentation combine random images together

# train_feats_comb = []
# train_labels_comb = []

# for i in range(5000):
#     to_combine = np.random.choice(range(len(train_feats)), size=2, replace=False)
#     a_train = train_feats[to_combine[0]]
#     b_train = train_feats[to_combine[1]]

#     a_label = train_labels[to_combine[0]]
#     b_label = train_labels[to_combine[1]]

#     c_train = np.max([a_train, b_train], 0)
#     c_label = np.max([a_label, b_label], 0)
    
#     train_feats_comb.append(c_train)
#     train_labels_comb.append(c_label)

In [ ]:
# train_feats_comb = np.array(train_feats_comb)
# train_labels_comb = np.array(train_labels_comb)

# train_feats_comb.shape, train_labels_comb.shape

In [ ]:
# plt.imshow(a_train[:, :, 0])
# plt.show()
# plt.imshow(b_train[:, :, 0])
# plt.show()
# plt.imshow(a_label[:, :, 0])
# plt.show()
# plt.imshow(b_label[:, :, 0])
# plt.show()
# plt.imshow(c_train[:, :, 0])
# plt.show()
# plt.imshow(c_label[:, :, 0])
# plt.show()

In [ ]:
# i = 3
# plt.imshow(a_train[:, :, i])
# plt.show()
# plt.imshow(b_train[:, :, i])
# plt.show()
# plt.imshow(a_label[:, :, i])
# plt.show()
# plt.imshow(b_label[:, :, i])
# plt.show()
# plt.imshow(c_train[:, :, i])
# plt.show()
# plt.imshow(c_label[:, :, i])
# plt.show()

In [ ]:
# get n random images from all images in the dataset
    
filename_g = f'models/datos_unet_gen_{loss}_{activation}_{today}'

for j in range(10000):
    
    # data augmentation combine
    comb = np.random.choice([0, 1, 1, 1])
    if comb == 1:
        
        train_feats_comb = []
        train_labels_comb = []

        for i in range(5000):
            to_combine = np.random.choice(range(len(train_feats)), size=2, replace=False)
            a_train = train_feats[to_combine[0]]
            b_train = train_feats[to_combine[1]]

            a_label = train_labels[to_combine[0]]
            b_label = train_labels[to_combine[1]]

            c_train = np.max([a_train, b_train], 0)
            c_label = np.max([a_label, b_label], 0)

            train_feats_comb.append(c_train)
            train_labels_comb.append(c_label)
            
        train_feats_comb = np.array(train_feats_comb)
        train_labels_comb = np.array(train_labels_comb)
        
        train_feats2 = train_feats_comb
        train_labels2 = train_labels_comb
        print('augmentation: combined random images')
    else:
        train_feats2 = train_feats
        train_labels2 = train_labels
        
    # data augmentation, flip vertically or horizontally
    flip = np.random.choice([0, 1, 2, 3, 4])
    if flip == 1: # horizontal flip
        
        train_feats2 = np.array([np.fliplr(i) for i in train_feats2])
        train_labels2 = np.array([np.fliplr(i) for i in train_labels2])
        print('flipped horizontally')
        
        k = np.random.choice([0, 1, 2, 3])
        if k != 0:

            train_feats2 = np.array([np.rot90(i, k=k) for i in train_feats2])
            train_labels2 = np.array([np.rot90(i, k=k) for i in train_labels2])
            
#             train_feats2 = np.array([rotate(i, k, reshape=False) for i in train_feats2])
#             train_labels2 = np.array([rotate(i, k, reshape=False) for i in train_labels2])
            print(f'rotated {k*90} degrees')
        
    elif flip == 2:

        train_feats2 = np.array([np.flipud(i) for i in train_feats2])
        train_labels2 = np.array([np.flipud(i) for i in train_labels2])
        print('flipped vertically')
        
        k = np.random.choice([0, 1, 2, 3])
        if k != 0:

            train_feats2 = np.array([np.rot90(i, k=k) for i in train_feats2])
            train_labels2 = np.array([np.rot90(i, k=k) for i in train_labels2])
            
#             train_feats2 = np.array([rotate(i, k, reshape=False) for i in train_feats2])
#             train_labels2 = np.array([rotate(i, k, reshape=False) for i in train_labels2])
            print(f'rotated {k*90} degrees')
            
    elif flip == 3:

        train_feats2 = np.array([np.fliplr(i) for i in train_feats2])
        train_labels2 = np.array([np.fliplr(i) for i in train_labels2])
        
        train_feats2 = np.array([np.flipud(i) for i in train_feats2])
        train_labels2 = np.array([np.flipud(i) for i in train_labels2])
        print('flipped vertically and horizontally')
        
        k = np.random.choice([0, 1, 2, 3])
        if k != 0:

            train_feats2 = np.array([np.rot90(i, k=k) for i in train_feats2])
            train_labels2 = np.array([np.rot90(i, k=k) for i in train_labels2])

#             train_feats2 = np.array([rotate(i, k, reshape=False) for i in train_feats2])
#             train_labels2 = np.array([rotate(i, k, reshape=False) for i in train_labels2])
            print(f'rotated {k*90} degrees')
            
    else:
#         train_feats2 = train_feats
#         train_labels2 = train_labels
        
        k = np.random.choice([0, 1, 2, 3])
        if k != 0:

            train_feats2 = np.array([np.rot90(i, k=k) for i in train_feats2])
            train_labels2 = np.array([np.rot90(i, k=k) for i in train_labels2])

#             train_feats2 = np.array([rotate(i, k, reshape=False) for i in train_feats2])
#             train_labels2 = np.array([rotate(i, k, reshape=False) for i in train_labels2])
            print(f'rotated {k*90} degrees')        
    # concatentate dataset
    dataset = [train_feats2, train_labels2]
    

    # train model
    train(unet_model, dataset, n_epochs=1, n_batch=1)

#     print(i)
    if j % 5 == 0:
        # save the generator model
        unet_model.save(f"{filename_g}_{j}.h5")
        print(f'saved generator model in {filename_g}_{j}.h5')


    # check on train
    for i in np.random.choice(list(range(len(dataset[0]))), 5):

        f, axs = plt.subplots(nrows=1, ncols=3, figsize=(5, 2))

        train_feat = train_feats2[i]
        train_true = train_labels2[i]
        train_true = compress_label(train_true)

        # show one input
        axs[0].imshow(train_feat[:, :, 0])
        
        # show true
        axs[1].imshow(train_true)

        # predict sample with values from 0 to 1
        train_pred = unet_model.predict(np.reshape(train_feat, (1, 128, 128, 10)))
        train_pred = compress_label(train_pred[0])

        axs[2].imshow(train_pred)

        plt.show()
        
        print(np.max(train_true), np.max(train_pred))
        print(np.min(train_true), np.min(train_pred))
        print(np.mean(train_true), np.mean(train_pred))

In [ ]:
# check on train
for i in np.random.choice(list(range(len(dataset[0]))), 5):

    f, axs = plt.subplots(nrows=1, ncols=3, figsize=(5, 2))

    train_feat = train_feats[i]
    train_true = train_labels[i]
    train_true = compress_label(train_true)

    # show one input
    axs[0].imshow(train_feat[:, :, 0])

    # show true
    axs[1].imshow(train_true)

    # predict sample with values from 0 to 1
    train_pred = unet_model.predict(np.reshape(train_feat, (1, 128, 128, 10)))
    train_pred = compress_label(train_pred[0])

    axs[2].imshow(train_pred)

    plt.show()

    print(np.max(train_true), np.max(train_pred))
    print(np.min(train_true), np.min(train_pred))
    print(np.mean(train_true), np.mean(train_pred))

In [ ]:
# # get n random images from all images in the dataset
# # for 128 X 128 X 1 output
    
# filename_g = f'models/datos_unet_gen_sigmoid_{loss}_{activation}_{today}'

# for i in range(10000):
    
#     # data augmentation, flip vertically or horizontally
#     flip = np.random.choice([0, 1, 2, 3, 4])
#     if flip == 1: # horizontal flip
#         print('flipped horizontally')
        
#         train_feats2 = np.array([np.fliplr(i) for i in train_feats])
#         train_labels2 = np.array([np.fliplr(i) for i in train_labels])
        
#         k = np.random.choice([0, 1, 2, 3])
#         if k != 0:
#             print(f'rotated {k*90} degrees')

#             train_feats2 = np.array([np.rot90(i, k=k) for i in train_feats2])
#             train_labels2 = np.array([np.rot90(i, k=k) for i in train_labels2])
        
#     elif flip == 2:
#         print('flipped vertically')
#         train_feats2 = np.array([np.flipud(i) for i in train_feats])
#         train_labels2 = np.array([np.flipud(i) for i in train_labels])
        
#         k = np.random.choice([0, 1, 2, 3])
#         if k != 0:
#             print(f'rotated {k*90} degrees')

#             train_feats2 = np.array([np.rot90(i, k=k) for i in train_feats2])
#             train_labels2 = np.array([np.rot90(i, k=k) for i in train_labels2])
        
#     elif flip == 3:
#         print('flipped vertically and horizontally')
#         train_feats2 = np.array([np.fliplr(i) for i in train_feats])
#         train_labels2 = np.array([np.fliplr(i) for i in train_labels])
        
#         train_feats2 = np.array([np.flipud(i) for i in train_feats2])
#         train_labels2 = np.array([np.flipud(i) for i in train_labels2])
        
#         k = np.random.choice([0, 1, 2, 3])
#         if k != 0:
#             print(f'rotated {k*90} degrees')

#             train_feats2 = np.array([np.rot90(i, k=k) for i in train_feats2])
#             train_labels2 = np.array([np.rot90(i, k=k) for i in train_labels2])
        
#     else:
#         train_feats2 = train_feats
#         train_labels2 = train_labels
        
#         k = np.random.choice([0, 1, 2, 3])
#         if k != 0:
#             print(f'rotated {k*90} degrees')

#             train_feats2 = np.array([np.rot90(i, k=k) for i in train_feats2])
#             train_labels2 = np.array([np.rot90(i, k=k) for i in train_labels2])
        
#     # concatentate dataset
#     dataset = [train_feats2, train_labels2]
    

#     # train model
#     train(unet_model, dataset, n_epochs=1, n_batch=1)

#     if i % 20 == 0:
#         # save the generator model
#         unet_model.save(f"{filename_g}_{i}.h5")
#         print(f'saved generator model in {filename_g}_{i}.h5')


#     # check on train
#     for i in np.random.choice(list(range(len(dataset[0]))), 5):

#         f, axs = plt.subplots(nrows=1, ncols=3, figsize=(5, 2))

#         train_feat = train_feats2[i]
#         train_true = train_labels2[i]

#         # show one input
#         axs[0].imshow(train_feat[:, :, 0])
        
#         # show true
#         axs[1].imshow(train_true)

#         # predict sample with values from 0 to 1
#         train_pred = unet_model.predict(np.reshape(train_feat, (1, 128, 128, 10)))

#         axs[2].imshow(train_pred[0][:, :, 0])

#         plt.show()
        
#         print(np.max(train_true), np.max(train_pred))

## Test Prediction

## Submission File